*Copyright 2019 StarkWare Industries Ltd.<br> Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.starkware.co/open-source-license/ <br> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.*

# Parte 1: La Traza y la Extensión de Bajo Grado (Low-Degree Extension)
Hoy desarrollaremos un probador STARK para la sucesión de los Cuadrados de Fibonacci sobre un finite field.
La sucesión de los Cuadrados de Fibonacci está definida por la relación de recurrencia $a_{n+2} = a_{n+1} ^2 + a_n ^2$.
<br>Al final del día, su código producirá una prueba *STARK* que demostrará la siguiente afirmación: <br>**Conozco un elemento del field $X\in \mathbb{F}$ tal que el 1023avo elemento de la secuencia de los Cuadrados de Fibonacci que comienza con $1$ y $X$ es $2338775057$**.
<br><br>
## Las Bases
### Clase FieldElement 
Usaremos nuestra clase `FieldElement` para representar elementos del field.<br> Puedes construir instancias de `FieldElement` a partir de enteros, y luego sumarlos, multiplicarlos, dividirlos, obtener inversos, etc.
El field subyacente de esta clase es $\mathbb{F}_{3221225473}$ ($3221225473 = 3 \cdot 2^{30} + 1$), todas las operaciones son hechas en módulo 3221225473.
<br><br>
Pruébalo ejecutando la siguiente celda (shift + enter):

In [ ]:
from field import FieldElement
FieldElement(3221225472) + FieldElement(10)

## Traza de la secuencia de los Cuadrados de Fibonacci

Para empezar, construiremos una lista `a` de longitud 1023, cuyos primeros dos elementos serán objetos FieldElement que representan al 1 y 3141592, respectivamente. Los siguientes 1021 elementos serán la secuencia de los Cuadrados de Fibonacci producida por esos dos elementos. `a` es como denominamos a la secuencia de los Cuadrados de Fibonacci, o, cuando el contexto es claro, la traza. <br>
Corrige el siguiente código para definir `a`:

In [ ]:
a = [FieldElement(1), FieldElement(3141592)]
'TU_CÓDIGO_AQUÍ'

### Prueba tu código
Ejecute la siguiente celda para comprobar que has definido `a` correctamente.<br> Note que esto se hace para verificar, aunque de manera ingenua y no sucinta, ya que repasa la secuencia elemento por elemento, que `a` se ha calculado de forma satisfactoria.

In [ ]:
assert len(a) == 1023, 'La traza debe constar exactamente de 1023 elementos.'
assert a[0] == FieldElement(1), 'El primer elemento en la traza debe ser el 1.'
for i in range(2, 1023):
    assert a[i] == a[i - 1] * a[i - 1] + a[i - 2] * a[i - 2], f'La regla de recursión de los Cuadrados de Fibonacci no se aplica al índice {i}'
assert a[1022] == FieldElement(2338775057), 'El último elemento es incorrecto!'
print('Éxito!')

## Pensando en Polinomios
Ahora queremos pensar en la secuencia como la evaluación de algún, todavía desconocido, polinomio $f$ de grado 1022 (debido al Unisolvence Theorem).
Elegiremos como dominio algún subgrupo $G \subseteq \mathbb{F}^\times$ de tamaño 1024, por razones que se aclararán más adelante.

(Recuerda que $\mathbb{F}^\times$ denota el grupo multiplicativo de $\mathbb{F}$, que obtenemos a partir de $\mathbb{F}$ omitiendo al cero, con la multiplicación presente en el field. Un subgrupo de tamaño 1024 existe porque $\mathbb{F}^\times$ es un grupo cíclico de tamaño $3\cdot 2^{30}$, de modo que contiene un subgrupo de tamaño $2^i$ para cualquier $0 \leq i \leq 30$).
### Encontrando un Grupo de Tamaño 1024
Si encontramos un elemento $g \in \mathbb{F}$ cuyo orden (multiplicativo) es 1024, entonces $g$ generará tal grupo.
La clase `FieldElement` proporciona un método estático `generator()` que retorna un elemento que genera $\mathbb{F}^\times$ (cuyo orden es $|\mathbb{F}^\times|$).
1. Úsalo para obtener un generador $g$ de $G$.
2. Crea una lista llamada `G` con todos los elementos de $G$, tal que $G[i] := g^i$.

*Pista: Cuando $k$ divide a $|\mathbb{F}^\times|$, $g^k$ genera un grupo de tamaño $\frac {|\mathbb{F}^\times|}{k}$, y la n-ésima potencia de algún `FieldElement` $x$ puede ser calculada llamando a `x ** n `.*



In [ ]:
# Cambia la siguiente línea para que g genere un grupo de tamaño 1024
g = FieldElement.generator()
# Defina G con los elementos de G tal que G[i] := g ** i
G = [] 

Ejecute la siguiente celda para probar tu código. 

In [ ]:
# Comprueba que g y G son correctos.
assert g.is_order(1024), 'El generador g es de orden incorrecto.'
b = FieldElement(1)
for i in range(1023):
    assert b == G[i], 'El lugar i-ésimo de G no es igual a la potencia i-ésima de g.'
    b = b * g
    assert b != FieldElement(1), f'g es de orden {i + 1}'
    
if b * g == FieldElement(1):
    print('Éxito!')
else:
    print('g es de orden > 1024')

### Clase Polynomial
Te proporcionaremos una clase llamada `Polynomial`. La forma más sencilla de construir un polinomio es usando la variable `X` (note que es una `X` mayúscula) que representa la variable formal $x$:

In [ ]:
from polynomial import X
# El polinomio 2x^2 + 1.
p = 2*X**2 + 1
# Evalúa p en 2:
print(p(2))
# Escribe un polinomio, por tu cuenta, en la última línea de la celda para visualizarlo
p

### Interpolando un Polinomio
Nuestro módulo `polynomial` proporciona una función de interpolación de Lagrange, cuyos argumentos son:
* x_values: correspondientes a los valores de x. [Lista]
* y_values: para los valores de y. [Lista]

Retorna el único `Polynomial` de grado < `len(x_values)` que evalúa a `y_values[i]` en `x_values[i]` para todos los valores de i.

Ejecute la siguiente celda para obtener ayuda sobre la función `interpolate_poly`.

In [ ]:
from polynomial import interpolate_poly
interpolate_poly?

Supongamos que `a` contiene los valores de algún polinomio sobre `G` (excepto `G[-1]`, ya que `a` es un elemento más corto).
Use `interpolate_poly()` para obtener `f` y conocer su valor en `FieldElement(2)`. 

In [ ]:
from polynomial import interpolate_poly
# Arregla lo siguiente para que v contenga el valor de f en FieldElement(2)
# Tenga en cuenta que interpolate_poly puede tardar hasta un minuto en ejecutarse.
v = 'TU_CÓDIGO_AQUÍ'

Ejecute la prueba:

In [ ]:
assert v == FieldElement(1302089273)
print('Éxito!')

## Evaluando en un dominio más grande
La traza, vista como evaluaciones de un polinomio $f$ en $G$, puede ampliarse ahora evaluando $f$ en un dominio mayor, creando así un código de corrección de errores Reed-Solomon.

### Cosets
Para ello, debemos dedicir un dominio mayor en el que $f$ será evaluado. 
Trabajaremos con un dominio que es 8 veces mayor que $G$. <br>Una elección natural para tal dominio es tomar algún grupo $H$ de tamaño 8192 (que existe porque 8192 divide a $|\mathbb{F}^\times|$), y cambiarlo por el generador de $\mathbb{F}^\times$, obteniendo así un [coset](https://en.wikipedia.org/wiki/Coset) de $H$.

Crea una lista `H` de los elementos de $H$, y multiplica cada uno de ellos por el generador de $\mathbb{F}^\times$ para obtener una lista llamada `eval_domain`. En otras palabras, eval_domain = $\{w\cdot h^i | 0 \leq i <8192  \}$ para $h$ el generador de $H$ y $w$ el generador de $\mathbb{F}^\times$.

Pista: Ya sabes cómo obtener $H$ - de forma similar a como obtuvimos $G$ hace unos minutos.


In [ ]:
# Arregla lo siguiente, asegúrate de que los elementos de H son potencias de su generador en 
# orden, es decir - H[0] será el 1, H[1] será h (el generador de H), H[2] será el generador de H
# al cuadrado, etc.
H = 'TU_CÓDIGO_AQUÍ'
eval_domain = 'TU_CÓDIGO_AQUÍ'

Ejecute la prueba:

In [ ]:
from hashlib import sha256
assert len(set(eval_domain)) == len(eval_domain)
w = FieldElement.generator()
w_inv = w.inverse()
assert '55fe9505f35b6d77660537f6541d441ec1bd919d03901210384c6aa1da2682ce' == sha256(str(H[1]).encode()).hexdigest(),\
    'La lista H es incorrecta. H[1] debería ser h (es decir, el generador de H).'
for i in range(8192):
    assert ((w_inv * eval_domain[1]) ** i) * w == eval_domain[i]
print('Éxito!')

### Evaluar en un Coset
Es hora de usar `interpolate_poly` y `Polynomial.poly` para evaluar sobre el coset. Tenga en cuenta que se implementa de forma ingenua en nuestro módulo de Python, por lo que la interpolación puede tardar hasta un minuto.<br>
De hecho, la interpolación y evaluación del polinomio de traza es uno de los pasos más intensos desde el punto de vista computacional del protocolo STARK, incluso cuando se utilizan métodos más eficientes (por ejemplo, FFT).

In [ ]:
# Define f_eval con las evaluaciones de f en eval_domain.
f_eval = 'TU_CÓDIGO_AQUÍ'

Ejecute la prueba:

In [ ]:
# Prueba usando un hash previamente calculado.
from hashlib import sha256
from channel import serialize
assert '1d357f674c27194715d1440f6a166e30855550cb8cb8efeb72827f6a1bf9b5bb' == sha256(serialize(f_eval).encode()).hexdigest()
print('Éxito!')

## Compromisos (Commitments)
Usaremos [Árboles de Merkle](https://en.wikipedia.org/wiki/Merkle_tree) basados en [Sha-256](https://en.wikipedia.org/wiki/SHA-2) como esquema de compromiso.
Una implementación sencilla de esto está disponible en la clase `MerkleTree`.
Ejecute la siguiente celda (por el bien de este tutorial, esto sirve como prueba de que todo el cálculo se ha ejecutado correctamente hasta este momento):

In [ ]:
from merkle import MerkleTree
f_merkle = MerkleTree(f_eval)
assert f_merkle.root == '6c266a104eeaceae93c14ad799ce595ec8c2764359d7ad1b4b7c57a4da52be04'
print('Éxito!')

## Channel
Teóricamente, un sistema de prueba STARK es un protocolo de interacción entre dos partes: un probador y un verificador. En la práctica, convertimos este protocolo interactivo en una prueba no interactiva utilizando la [Heurística Fiat-Shamir](https://en.wikipedia.org/wiki/Fiat%E2%80%93Shamir_heuristic). En este tutorial se utilizará la clase `Channel`, que implementa esta transformación. Este channel reemplaza al verificador en el sentido de que el probador enviará datos, y recibirá números aleatorios o instancias aleatorias de FieldElement.

Este simple trozo de código instanciará un channel object y envía la raíz de su Árbol de Merkle. 
Luego, el channel object puede ser llamado para proporcionar números aleatorios o elementos aleatorios del field.

In [ ]:
from channel import Channel
channel = Channel()
channel.send(f_merkle.root)

Finalmente, puedes recuperar la prueba hasta el momento (es decir, todo lo que ha pasado por el channel hasta cierto punto) imprimiendo `Channel.proof`.

In [ ]:
print(channel.proof)